# 1. Presets

1. Run setup notebook!
2. Add Slim modules to Python Path

In [ ]:
!pip install contextlib2
import os
slim_path = ":models/research/slim"
old_path = (os.environ.get("PYTHONPATH") or '')

if not (slim_path in old_path):
    new_python_path = old_path + slim_path

%env PYTHONPATH=$new_python_path

# 2. Download and build visual wakeword dataset

The existing dataset (download via the setup notebook) is designed to be used for training models for localization, so the images aren't labeled with the "contains a X", "doesn't contain a X" categories that we want to train for. Instead each image comes with a list of bounding boxes for all of the objects it contains. "X" is one of these object categories, so to get to the classification labels we want, we have to look for images with bounding boxes for people. To make sure that they aren't too tiny to be recognizable we also need to exclude very small bounding boxes. Slim contains a script to convert the bounding box into labels:

In [ ]:
LABEL='person'
%cd ~
! python sparky-rooms/tensorflow-model/models/research/slim/download_and_convert_data.py \
--dataset_name=visualwakewords \
--dataset_dir=dataset/visualwakewords \
--small_object_area_threshold=0.005 \
--foreground_class_of_interest=$LABEL

# 3. Train

"train_image_classifier.py" only works with TF1.x

In [ ]:
pip install "tensorflow>=1.15,<2.0"

In [16]:
import tensorflow as tf
print(tf.__version__)

1.15.4


In [18]:
%cd ~
! python sparky-rooms/tensorflow-model/models/research/slim/train_image_classifier.py \
    --train_dir=vww_96_grayscale \
    --dataset_name=visualwakewords \
    --dataset_split_name=train \
    --dataset_dir=dataset/visualwakewords \
    --model_name=mobilenet_v1_025 \
    --preprocessing_name=mobilenet_v1 \
    --train_image_size=96 \
    --input_grayscale=True \
    --save_summaries_secs=300 \
    --learning_rate=0.045 \
    --label_smoothing=0.1 \
    --learning_rate_decay_factor=0.98 \
    --num_epochs_per_decay=2.5 \
    --moving_average_decay=0.9999 \
    --batch_size=96 \
    --max_number_of_steps=1000000

/home/jupyter
Instructions for updating:
Please switch to tf.train.create_global_step
W1103 11:04:45.515206 140702958704000 deprecation.py:323] From sparky-rooms/tensorflow-model/models/research/slim/train_image_classifier.py:430: create_global_step (from tf_slim.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W1103 11:04:45.548903 140702958704000 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tf_slim/data/parallel_reader.py:249: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced 

# 4. Eval

In [ ]:
! python sparky-rooms/tensorflow-model/models/research/eval_image_classifier.py \
--alsologtostderr checkpoint_path=vww_96_grayscale/model.ckpt-1000000 \
--dataset_dir=coco_dataset \
--dataset_name=visualwakewords \
--dataset_split_name=val \
--model_name=mobilenet_v1_025 \
--preprocessing_name=mobilenet_v1 \
--use_grayscale=True \
--train_image_size=96 \